In [1]:
from task import *

/data/mn27889/miniconda3/envs/mental-health-agents/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


path is  /data/mn27889/mental-health-agents/src/sdoh_analysis/helpers/../models/setfit-mental-health-topic-prediction/setfit-small-research-topic-prediction/content/setfit-small-research-topic-prediction
True


/data/mn27889/miniconda3/envs/mental-health-agents/lib/python3.12/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/data/mn27889/miniconda3/envs/mental-health-agents/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  w

encoding database


2775it [00:00, 476957.49it/s]


got 2511 of 2749 others out bc of size


100%|██████████| 883/883 [00:00<00:00, 314234.72it/s]

encodnig databsae - this can take a while - TODO - cache this


db encoded


In [2]:
import re

In [3]:
task = MentalHealthTask()

Repo card metadata block was not found. Setting CardData to empty.


In [4]:
idx = 2
x = task.get_input(idx)

In [5]:
print(x)

 really want to know what’s wrong with me.

Socially I’m scared and very judging of myself. Particularly with regards to my emotional expression, I feel I don’t have emotions socially, or they’re not appropriate. I’m awkward and really want the approval of others to know I’m doing okay and they like me. I don’t care who it’s from. I don’t want friends, I just want approval. However I deny myself it because it feels narcissistic, so I’m just quiet. I don’t want to be burdensome. I have no relationship with my parents. I don’t want to fall in love with anyone who’s not my partner because I don’t want to ruin it when it finally happens, I want to save myself for it. However I have never had a partner and may not for many years due to reasons I don’t know. However having a partner is the most important thing in life to me, yet I’m shallow enough to care a lot about their physical appearance.

I have very few real interests or interest in doing things that don’t involve women or thinking ab

In [6]:
determinant_prompt, example_format, sdoh_prompt = task.get_cot_sdoh_prompt_data(input=x, idx=idx)

In [7]:
final_prompt = user_prompt_persona.format(user_input=x, user_past_history=sdoh_prompt)

In [8]:
print(final_prompt)

You are given a user input shared by a mentally ill person along with the past history of the same person categorized by social determinants.
Considering the user input and the past history of the user, your job is to determine 3 therapy techniques out of the following 8 therapy techniques
to better improve the overall mental health of the person: (1) Cognitive Behavioral Therapy (2) Exposure Therapy (3) Psychodynamic Therapy (4) Client Centered Therapy
(5) Humanistic Therapy (6) Interpersonal Therapy (7) Mentalization Therapy (8) Mindfulness Therapy.

User Input:  really want to know what’s wrong with me.

Socially I’m scared and very judging of myself. Particularly with regards to my emotional expression, I feel I don’t have emotions socially, or they’re not appropriate. I’m awkward and really want the approval of others to know I’m doing okay and they like me. I don’t care who it’s from. I don’t want friends, I just want approval. However I deny myself it because it feels narcissist

In [9]:
output = gpt(user_prompt=final_prompt, system_prompt=sys_prompt_persona)
answer = output[0]
print(answer)

Therapy Technique: Cognitive Behavioral Therapy | Interpersonal Therapy | Mindfulness Therapy


In [10]:
match = re.match(r".*Therapy Technique:([\s\|\w]+).*", answer, re.DOTALL)

In [11]:
therapy_techniques = match.groups()[0]
therapy_list = therapy_techniques.split('|')
therapy_list = [therapy.strip() for therapy in therapy_list]

In [14]:
final_counsel = []
for therapy in therapy_list:
    final_prompt_therapy = user_prompt_therapy.format(therapy_technique=therapy, user_input=x, user_past_history=sdoh_prompt)
    therapy_response = gpt(user_prompt=final_prompt_therapy, temperature=0.7, n=3)

    extracted_counsel = []
    for index, out in enumerate(therapy_response):
        match_counsel = re.match(r".*Therapy Counsel.*:\n*(.*)", out, re.DOTALL)
        extracted_counsel.append(match_counsel.groups()[0])
    
    vote_choice = ""
    for index, counsel in enumerate(extracted_counsel):
        vote_choice += "Choice " + str(index+1) + ":\n\n" + counsel + "\n\n"
        
    final_prompt_vote = user_vote_prompt.format(user_input=x, user_past_history=sdoh_prompt, counselling_choices=vote_choice)
    vote_score = gpt(user_prompt=final_prompt_vote, system_prompt=system_vote_prompt, temperature=0.7, n=1)
    match_vote_score = re.match(r".*best choice is .*(\d+).*", vote_score[0], re.DOTALL)
    
    print(extracted_counsel[int(match_vote_score.groups()[0])-1])
    final_counsel.append(extracted_counsel[int(match_vote_score.groups()[0])-1])

I understand that you're going through a lot right now. It seems like you're struggling with feelings of self-doubt, guilt, and fear of social situations. It's okay to feel this way, and I want to assure you that you're not alone.

Firstly, it's important to understand that everyone has their unique pace and way of experiencing emotions, and it's okay if yours are different from others. It doesn't mean that you're less real or less human. Your feelings and experiences are valid, and it's okay to express them.

You've mentioned that you constantly seek approval from others, which tells me that you might be doubting your self-worth. It's essential to understand that your value doesn't depend on others' approval. Engaging in activities that you enjoy can help boost your self-esteem and reduce these feelings over time.

You also seem to have some fears around social situations. It's okay to feel anxious, but remember that everyone makes mistakes in social settings, and it's a part of learn

In [15]:
final_counsel

["I understand that you're going through a lot right now. It seems like you're struggling with feelings of self-doubt, guilt, and fear of social situations. It's okay to feel this way, and I want to assure you that you're not alone.\n\nFirstly, it's important to understand that everyone has their unique pace and way of experiencing emotions, and it's okay if yours are different from others. It doesn't mean that you're less real or less human. Your feelings and experiences are valid, and it's okay to express them.\n\nYou've mentioned that you constantly seek approval from others, which tells me that you might be doubting your self-worth. It's essential to understand that your value doesn't depend on others' approval. Engaging in activities that you enjoy can help boost your self-esteem and reduce these feelings over time.\n\nYou also seem to have some fears around social situations. It's okay to feel anxious, but remember that everyone makes mistakes in social settings, and it's a part 

In [16]:
counsellor_advice_choices = ""
for index, advice in enumerate(final_counsel):
    counsellor_advice_choices += "Advice " + str(index+1) + ":\n\n" + advice + "\n\n"

final_prompt_summarize = user_summarize_prompt.format(user_input=x, user_past_history=sdoh_prompt, counselling_advices=counsellor_advice_choices)
summarized_response = gpt(user_prompt=final_prompt_summarize, system_prompt=system_summarize_prompt, temperature=0.7, n=1)

In [18]:
print(summarized_response[0])

I'm sorry to hear you're feeling this way, but please know that it's okay to feel as you do and you're not alone. Your struggle with self-doubt, guilt, and fear of social situations is valid, and it's good that you're reaching out to understand your feelings better.

Based on the advice from multiple counsellors, it seems that you are having a hard time with self-acceptance and you often seek approval from others. Remember, everyone's emotional experience is unique and your feelings are as valid as anyone else's. It's important to understand that your worthiness is not contingent on others' approval. You might find it beneficial to engage in activities you enjoy, as doing so can help improve your self-esteem and shift these feelings over time.

You've expressed fear of social situations. Remember, it's okay to make mistakes in social interactions; they're part of the learning process. You can gradually build your social skills and confidence with practice. Mindfulness techniques, such 

### Testing Code

In [ ]:
final_prompt_therapy = user_prompt_therapy.format(therapy_technique=therapy_list[1], user_input=x, user_past_history=sdoh_prompt)

In [ ]:
print(final_prompt_therapy)

In [ ]:
output = gpt(user_prompt=final_prompt_therapy, temperature=0.7, n=3)

In [ ]:
output

In [ ]:
example_output = output

In [ ]:
extracted_counsel = []
for index, out in enumerate(example_output):
    match_counsel = re.match(r".*Therapy Counsel.*:\n*(.*)", out, re.DOTALL)
    extracted_counsel.append(match_counsel.groups()[0])

In [ ]:
vote_choice = ""
for index, out in enumerate(extracted_counsel):
    vote_choice += "Choice " + str(index+1) + ":\n\n" + out + "\n\n"

In [ ]:
print(vote_choice)

In [ ]:
final_prompt_vote = user_vote_prompt.format(user_input=x, user_past_history=sdoh_prompt, counselling_choices=vote_choice)

In [ ]:
output = gpt(user_prompt=final_prompt_vote, system_prompt=system_vote_prompt, temperature=0.7, n=1)

In [ ]:
output

In [ ]:
pattern = r".*best choice is .*(\d+).*"
match = re.match(pattern, output[0], re.DOTALL)

In [ ]:
sel_counsel = extracted_counsel[int(match.groups()[0])-1]

In [ ]:
print(sel_counsel)

In [ ]:
cognitive_response = '''
I understand that you're feeling lost, disconnected, and critical of yourself right now. It's important to remember that it's okay to feel these things, and it's okay to ask for help.

Firstly, let's work on building self-compassion. You're very hard on yourself, and it's crucial to learn to treat yourself with the same kindness you'd extend to others. There are exercises we can work through together to help you build this skill.

Secondly, your thought patterns seem to be dominated by negative perceptions of yourself. Cognitive Behavioral Therapy (CBT) can be very effective in identifying and challenging these thoughts, and we can use cognitive restructuring exercises to help you with this.

Social situations appear to cause you a lot of distress. We can gradually expose you to these situations in a controlled manner to help reduce your fear over time. Remember, it's okay to be yourself and not everyone needs to approve of you. Your worth is not defined by others' approval.

Your need for approval and your feelings about your physical appearance suggest low self-esteem. Building a positive self-image independent of others can be empowering and freeing. 

Your struggle with emotional expression is something we can address with emotional intelligence training. This will help you understand your emotions and express them appropriately.

You've mentioned that you have very few interests. Exploring different activities might help you find something you're passionate about, which can bring joy and purpose into your life.

Your difficulties with planning and losing things suggest that working on your organizational skills could be beneficial. This can help reduce anxiety and feelings of being overwhelmed.

Finally, incorporating mindfulness and relaxation techniques into your daily routine can help manage feelings of guilt and anxiety, and improve focus and observation skills. 

Remember, improvement takes time and patience. It's okay to take small steps. You're not alone in this journey, and I'm here to support you along the way.
'''

In [ ]:
interpersonal_therapy = '''
I understand that you're feeling lost and scared socially, and are struggling with your self-perception. It's okay to feel this way, and it's a step forward that you're acknowledging these feelings. In interpersonal therapy, we focus on the relationships that you have with others, including the one with yourself. 

It seems like you're seeking approval from others, but it's also important to seek approval from yourself. It's not narcissistic to want to feel good about yourself. Instead of focusing on how others perceive you, let's work on how you perceive yourself. 

Your relationship with your parents and your fear of falling in love seem to be significant factors in your life. Let's explore these feelings in more depth, and understand how they're affecting your sense of self and interactions with others. 

You've mentioned that you feel unobservant and impulsive, which can be unsettling. We can work on mindfulness exercises that can help you become more aware of your surroundings and reactions to situations. 

I understand that you're feeling guilty about not doing more or being a better person. Remember, it's okay to not have all the answers and it's okay to just be. You're not alone, and together, we can navigate these feelings and work towards improving your mental health. 

Lastly, about your interests, it's okay to not have many interests or hobbies. However, exploring new ones could be a great way for you to connect with others and yourself. Let's start by considering things you've always wanted to do or try, and take it from there.

Remember, it's okay to reach out and ask for help. You're not alone in this journey.
'''

In [ ]:
mindfulness_therapy = '''
I understand that you're feeling lost, and it's okay to feel this way. It's important to remember that everyone has their own unique journey and it's perfectly okay to not have everything figured out.

You mentioned feeling scared and judgmental of yourself socially, and that you crave approval. It's not narcissistic to desire approval, it's human. What we'll work on is helping you find a sense of self-approval and self-love, so that you're less reliant on external validation.

You also expressed a sense of disconnection from your emotions. Through mindfulness exercises, we'll work on helping you reconnect with your feelings. Mindfulness can help you become an observer of your emotions, rather than feeling overwhelmed or disconnected from them.

Regarding your interests, it's okay to not have a long list. We can explore together some activities that might bring you joy and satisfaction. Remember, it's not about doing what everyone else is doing, it's about finding what brings you happiness.

In terms of relationships, it's important to remember that it's okay to be single. A partner can be a wonderful addition to life, but it's equally important to feel confident and happy on your own. We can work on developing healthy expectations and boundaries in relationships.

Lastly, you mentioned feeling unobservant and impulsive. We'll explore cognitive behavioral techniques to help you challenge these patterns of thinking and behaving, and develop more effective strategies for dealing with real-world scenarios. 

Remember, it's okay to feel the way you do. You're not alone, and there is help available for you. It's a brave step to reach out, and I encourage you to keep going. You're much stronger than you think.
'''

In [ ]:
counsellor_advices = [cognitive_response, interpersonal_therapy, mindfulness_therapy]

In [ ]:
counsellor_advice_choices = ""
for index, advice in enumerate(counsellor_advices):
    counsellor_advice_choices += "Advice " + str(index+1) + ":\n\n" + advice + "\n\n"

In [ ]:
final_prompt_summarize = user_summarize_prompt.format(user_input=x, user_past_history=sdoh_prompt, counselling_advices=counsellor_advice_choices)

In [ ]:
print(final_prompt_summarize)

In [ ]:
output = gpt(user_prompt=final_prompt_summarize, system_prompt=system_summarize_prompt, temperature=0.7, n=1)

In [ ]:
print(output[0])